<a href="https://colab.research.google.com/github/junxnone/examples/blob/master/pytorch/pytorch_LeNet_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import torchvision
from torch.autograd import Variable
from torch.utils.data import DataLoader
import cv2

# 定义 LeNet

In [0]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.BatchNorm1d(120),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.ReLU()
        )

        self.fc3 = nn.Linear(84, 10)

    #         self.sfx = nn.Softmax()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        #         print(x.shape)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        #         x = self.sfx(x)
        return x

# 预处理数据

In [0]:
batch_size = 64

train_dataset = datasets.MNIST(root = './data/',
                              train=True,
                              transform = transforms.ToTensor(),
                              download=True)
test_dataset =datasets.MNIST(root = './data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)

# 定义Model

In [0]:
LR = 0.001
Momentum = 0.9
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=LR,momentum=Momentum)

# Training

In [5]:
epoch = 1

for epoch in range(epoch):
    sum_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print(loss)
        sum_loss += loss.item()
        if i % 100 == 99:
            print('[%d,%d] loss:%.03f' % (epoch + 1, i + 1, sum_loss / 100))
            sum_loss = 0.0

[1,100] loss:1.551
[1,200] loss:0.761
[1,300] loss:0.495
[1,400] loss:0.365
[1,500] loss:0.303
[1,600] loss:0.255
[1,700] loss:0.225
[1,800] loss:0.196
[1,900] loss:0.184


# Test

In [6]:
net.eval()
correct = 0
total = 0
for data_test in test_loader:
    images, labels = data_test
    images, labels = Variable(images).cuda(), Variable(labels).cuda()
    output_test = net(images)
    # print("output_test:",output_test.shape)

    _, predicted = torch.max(output_test, 1)
    # print("predicted:",predicted.shape)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print("correct1: ",correct)
print("Test acc: {0}".format(correct.item() / len(test_dataset)))#.cpu().numpy()

correct1:  tensor(9688, device='cuda:0')
Test acc: 0.9688
